# evaluation_digital_end2end.ipynb

This notebook is used to evalute the end2end solution on digital digit data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import glob
import numpy as np
test_dir = 'drive/My Drive/Team Bergere/Optimizer/Data/optimizer/test_set/'

# **Digital Images**

In [5]:
#@title
import numpy as np
import os
# import helper
# import detection
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import glob

Using TensorFlow backend.


In [0]:
hq_digital = pd.read_excel(test_dir + 'HQ_digital_test/hq_digital_test.xls', dtype = {'label':str})
hq_digital.columns = ['file_name', 'label']
hq_digital['quality'] = 'H'
mq_digital = pd.read_csv(test_dir + 'MQ_digital_test/mq_test_label.csv', delimiter = ';',dtype = {'label':str})
mq_digital.columns = ['file_name', 'label']
mq_digital['quality'] = 'M'
lq_digital = pd.read_excel(test_dir + 'LQ_digital_test/lq_digital_test.xls',  dtype = {'label':str})
lq_digital.columns = ['file_name', 'label']
lq_digital['quality'] = 'L'

digital_labels = pd.concat([hq_digital, mq_digital,lq_digital], axis = 0)

In [0]:
## Digital pictures
digital_pictures = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/test_set/*digital*/*.jpg")
digital_pictures_name = [x.split("/") for x in digital_pictures]
digital_pictures_name = [x[len(x)-1] for x in digital_pictures_name]
df_digital = pd.DataFrame(
                    {'files': digital_pictures,
                     'file_name':digital_pictures_name
                    })

In [8]:
df_digital.head()

,files,file_name
0,/content/drive/My Drive/Team Bergere/Optimizer...,1bc7bbefa14f6d7680e055191f8ba99823defba6.jpg
1,/content/drive/My Drive/Team Bergere/Optimizer...,3dda11f98b920b17f0cb011cb881888801bf991a.jpg
2,/content/drive/My Drive/Team Bergere/Optimizer...,1f27a79030d2b43055f03b2c77214f1c29681dcd.jpg
3,/content/drive/My Drive/Team Bergere/Optimizer...,0fc1317374529ccc9ead94895c8722c1681ce187.jpg
4,/content/drive/My Drive/Team Bergere/Optimizer...,2b1f5ea851600eec2cc600014ffd59a320c096e3.jpg


In [0]:
def prepareLabelDataset(df_digital, df_label = digital_labels):
  """
  This function is used to prepare the dataframe with different columns that
  can be used to calculate different metrics.
  
  Arguments:
  df_digital -- a dataframe with columns 'files' and 'filesname'
  df_label -- a dataframe with columns 'image' and 'label'
  
  Returns:
  df_digital -- a dataframe with columns 'file_name', 'label', ..., 'weights'
  
  """
  
  df_digital = df_digital.merge(df_label,
                              on = 'file_name',
                              how = 'left')
  df_digital['original_label'] = df_digital['label']
  df_digital = df_digital.loc[~df_digital['label'].isna()]
  ####Geting the correct label
  df_digital['len_label'] = df_digital['label'].apply(str).apply(len)
  pictures_to_take = df_digital['len_label']>3
  df_digital = df_digital.loc[df_digital['len_label']>3]
  df_digital['len_label_decimal'] =  df_digital['label'].apply(lambda x: len(x.split('.')[1]))
  df_digital['label'] = np.where(df_digital['label'] == 1, df_digital.label + "x", df_digital.label)

  def padding(s):
    length = len(str(s))
    return (5-length)*"x" +str (s)
  #train["padded"] = train["correct_label"].apply(padding)
  df_digital["label"] = df_digital['label'].apply(str).apply(lambda x: x.replace(".", ""))
  df_digital['label'] = df_digital["label"].apply(padding)

  df_digital['final_label'] = (df_digital['len_label'] -1).astype(str) +df_digital['label']
  df_digital["final_label"] = df_digital['final_label'].apply(list).apply(lambda a: ["10" if x=="x" else x for x in a])
  
  
  ## Process image
  def processImage(i):
    target_size = (224,224)
    img = image.load_img(i)
    img = img.resize(target_size)
    x = image.img_to_array(img)
    return x
  
  df_digital['pictures'] = df_digital['files'].apply(processImage)
  df_digital['weights'] = df_digital['original_label'].apply(len)-1
#   del df_digital['pictures']
  del df_digital['files']
  
  return df_digital

In [10]:
df_digital_all = prepareLabelDataset(df_digital = df_digital, df_label = digital_labels)
df_digital_all.head()

,file_name,label,quality,original_label,len_label,len_label_decimal,final_label,pictures,weights
0,1bc7bbefa14f6d7680e055191f8ba99823defba6.jpg,x5500,H,55.00,5,2,"[4, 10, 5, 5, 0, 0]","[[[135.0, 141.0, 131.0], [142.0, 137.0, 131.0]...",4
1,3dda11f98b920b17f0cb011cb881888801bf991a.jpg,x5400,H,54.00,5,2,"[4, 10, 5, 4, 0, 0]","[[[180.0, 181.0, 176.0], [183.0, 184.0, 179.0]...",4
2,1f27a79030d2b43055f03b2c77214f1c29681dcd.jpg,15000,H,150.00,6,2,"[5, 1, 5, 0, 0, 0]","[[[51.0, 28.0, 14.0], [53.0, 30.0, 16.0], [45....",5
3,0fc1317374529ccc9ead94895c8722c1681ce187.jpg,x6600,H,66.00,5,2,"[4, 10, 6, 6, 0, 0]","[[[189.0, 193.0, 192.0], [189.0, 193.0, 192.0]...",4
4,2b1f5ea851600eec2cc600014ffd59a320c096e3.jpg,12000,H,120.00,6,2,"[5, 1, 2, 0, 0, 0]","[[[158.0, 163.0, 157.0], [166.0, 161.0, 158.0]...",5


In [0]:
def modelPrediction(img, model):
  """
  This function is used to predict the output
  
  Arguments:
  
  img -- an image array used to input the model
  model -- the model to predict output
  
  Returns:

  a list with five elements:
  
  numDig -- number of digits in the picture
  d1 -- the first digits
  d2 -- the second digits
  d3 -- the third digits
  d4 -- the forth digits
  d5 -- the fifth digits
  
  """
  yOut = model.predict(img.reshape(1,224,224,3))
  numDig = np.argmax(yOut[0].squeeze())
  d1 = np.argmax(yOut[1].squeeze())
  d2 = np.argmax(yOut[2].squeeze())
  d3 = np.argmax(yOut[3].squeeze())
  d4 = np.argmax(yOut[4].squeeze())
  d5 = np.argmax(yOut[5].squeeze())
  return [numDig, d1, d2, d3,d4, d5]

# **Analog Images**

In [0]:
hq_analog = pd.read_excel(test_dir + 'HQ_analog_test/hq_analog_test.xls')

In [0]:
hq_analog.head()

,image,label
0,00b4df21fc7774ebe26658797c5e78f1e57ac05c.jpg,110
1,00d4904613d2aa43a2d911fa77a8ae00f8a64fac.jpg,311
2,00daab3e561e37286be49ed197bf5aaba589fafe.jpg,161
3,00ec9bd05f6d3b9771526d4190ac99a0cd5cfddc.jpg,48
4,0a03ff6ef4661817813387f347627373602efbea.jpg,132


# **Evaluation**

## 1. Digit-by-digit accuracy

In [0]:
def digit_by_digit_accuracy(df_label, df_prediction):
  
  """
  This function is used to calculate the digit by digit accuracy
  
  """
  
  ## Remove len
  label = np.array(df_label[1:])
  prediction = np.array(df_prediction[1:])
  ## Compute how many digits are equal
  label = np.array(label).astype(str)
  prediction = np.array(prediction).astype(str)
  return (label == prediction).sum()/len(label)

## 2. Multi-class accuracy

In [0]:
def multi_class_accuracy(df_label, df_prediction):
  
  raise NotImplementedError

def multi_class_accuracy(df_label, df_prediction):
  raise NotImplementedError

## 3. Summation accuracy or Mean Absolute Error rate

In [0]:
def total_consumption(df, col = 'label'):
  """
  This function is used to calculate the total consumption
  """
  return df[col].sum()

In [0]:
def getOriginalNumber(df_label):
  """
  This function is used to transform output label to final digits
  """
  label = df_label[1:]
  [str(i) for i in label]
  label = ["0" if x=="10" or x == 10 else x for x in label]
  label_str = str(label[0]) + str(label[1]) + str(label[2]) + "." + str(label[3])+ str(label[4])
  return float(label_str)  
 
def mean_absolute_error(df_label, df_prediction):
  """
  This function is used to calculate the absolute error
  """
  label = getOriginalNumber(df_label)
  prediction = getOriginalNumber(df_prediction)
  return np.abs(label- prediction)

def summation_accuracy(df_label, df_prediction):
  """
  This function is used to calculate the summation accuracy
  """
  label = getOriginalNumber(df_label)
  prediction = getOriginalNumber(df_prediction)
  return prediction - label

## Model by model comparison

In [0]:
model_20k_85 = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/*model_v4.85*")[0]
model_20k_85

'/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/model_v4.85.h5'

### Overall digital test set

In [0]:
def evaluate(df, model):
  """
  This function is used to calculate different metrics
  
  Arguments:
  
  df -- a dataframe with the image name and label
  model -- the model used to predict
  
  Returns:

  df_digital -- a dataframe with the predicted value and metrics  
  """
  

  df_digital = df.copy()
  model = load_model(model)
  df_digital['prediction'] = df_digital['pictures'].apply(modelPrediction,model = model)
  df_digital['digit_by_digit'] = df_digital[['final_label','prediction']].apply(lambda x: digit_by_digit_accuracy(x.iloc[0], x.iloc[1]), axis = 1)
  print('digit_by_digit_accuracy', df_digital['digit_by_digit'].mean())
  df_digital['absolute_error'] =  df_digital[['final_label','prediction']].apply(lambda x: mean_absolute_error(x.iloc[0], x.iloc[1]), axis = 1)
  print('mean_absolute_error', df_digital['absolute_error'].mean())
  df_digital['multi_class'] = df_digital['digit_by_digit']*df_digital['weights']
  print('weighted_digit_by_digit_accuracy',sum(df_digital['multi_class'])/sum(df_digital['weights']) )
  df_digital['difference'] =  df_digital[['final_label','prediction']].apply(lambda x: summation_accuracy(x.iloc[0], x.iloc[1]), axis = 1)
  diff = df_digital['difference'].sum()
  print("summation difference : prediction - origin", diff)
  df_digital['final_prediction'] = df_digital['final_label'].apply(getOriginalNumber)
  allcon = df_digital['final_prediction'].sum()
  print("total origin consumption", allcon)
  print("Overall error rate", diff/ allcon )
  
  del model
  del df_digital['pictures']
  del df_digital['len_label']
  del df_digital['len_label_decimal']
  del df_digital['weights']

  
  return(df_digital)

In [0]:
df_all_20k = evaluate(df_digital_all,model_20k_85 )






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
digit_by_digit_accuracy 0.6235294117647057
mean_absolute_error 41.08400000000001
weighted_digit_by_digit_accuracy 0.622043010752688
summation difference : prediction - origin 117.87999999999997
total origin consumption 9271.15
Overall error rate 0.012714711767148624


In [0]:
model_10k = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/model_good.h5")[0]
df_all_10k = evaluate(df_digital_all,model_10k )

digit_by_digit_accuracy 0.6399999999999998
mean_absolute_error 37.62305882352942
weighted_digit_by_digit_accuracy 0.6392473118279568
summation difference : prediction - origin -85.55999999999986
total origin consumption 9271.15
Overall error rate -0.009228628595158083


## HQ data

In [0]:
df_hq = prepareLabelDataset(df_digital = df_digital, df_label = hq_digital)
df_hq.head()

,file_name,label,quality,original_label,len_label,len_label_decimal,final_label,pictures,weights
0,1bc7bbefa14f6d7680e055191f8ba99823defba6.jpg,x5500,H,55.00,5,2,"[4, 10, 5, 5, 0, 0]","[[[135.0, 141.0, 131.0], [142.0, 137.0, 131.0]...",4
1,3dda11f98b920b17f0cb011cb881888801bf991a.jpg,x5400,H,54.00,5,2,"[4, 10, 5, 4, 0, 0]","[[[180.0, 181.0, 176.0], [183.0, 184.0, 179.0]...",4
2,1f27a79030d2b43055f03b2c77214f1c29681dcd.jpg,15000,H,150.00,6,2,"[5, 1, 5, 0, 0, 0]","[[[51.0, 28.0, 14.0], [53.0, 30.0, 16.0], [45....",5
3,0fc1317374529ccc9ead94895c8722c1681ce187.jpg,x6600,H,66.00,5,2,"[4, 10, 6, 6, 0, 0]","[[[189.0, 193.0, 192.0], [189.0, 193.0, 192.0]...",4
4,2b1f5ea851600eec2cc600014ffd59a320c096e3.jpg,12000,H,120.00,6,2,"[5, 1, 2, 0, 0, 0]","[[[158.0, 163.0, 157.0], [166.0, 161.0, 158.0]...",5


In [0]:
df_hq_20k = evaluate(df_hq,model_20k_85 )

digit_by_digit_accuracy 0.5416666666666667
mean_absolute_error 49.291666666666664
weighted_digit_by_digit_accuracy 0.54
summation difference : prediction - origin -514.3599999999999
total origin consumption 3334.9700000000003
Overall error rate -0.1542322719544703


In [0]:
df_hq_10k = evaluate(df_hq,model_10k )

digit_by_digit_accuracy 0.55
mean_absolute_error 45.73
weighted_digit_by_digit_accuracy 0.5509090909090909
summation difference : prediction - origin -569.88
total origin consumption 3334.9700000000003
Overall error rate -0.17088009787194486


## MQ digital data

In [0]:
df_mq = prepareLabelDataset(df_digital = df_digital, df_label = mq_digital)
df_mq.head()

,file_name,label,quality,original_label,len_label,len_label_decimal,final_label,pictures,weights
60,0b10bbdc8da66f705ae2a62c72fcb6a88bd2a79d.jpg,17107,M,171.07,6,2,"[5, 1, 7, 1, 0, 7]","[[[254.0, 254.0, 254.0], [254.0, 254.0, 254.0]...",5
61,0a6e4d5f38965dc3d805720f4f0886183d27040c.jpg,x2300,M,23.00,5,2,"[4, 10, 2, 3, 0, 0]","[[[160.0, 175.0, 182.0], [160.0, 175.0, 182.0]...",4
62,00e22973fdbc19f1f703cdbd05a8606705bc60d1.jpg,23000,M,230.00,6,2,"[5, 2, 3, 0, 0, 0]","[[[240.0, 252.0, 248.0], [240.0, 252.0, 248.0]...",5
63,0c31f652e512dae07bee74292516fa0e089adb67.jpg,x5900,M,59.00,5,2,"[4, 10, 5, 9, 0, 0]","[[[254.0, 254.0, 254.0], [254.0, 254.0, 254.0]...",4
64,0bdaeea0d984ff3b500fc1360b18e57deae694ca.jpg,x1500,M,15.00,5,2,"[4, 10, 1, 5, 0, 0]","[[[254.0, 254.0, 254.0], [254.0, 254.0, 254.0]...",4


In [0]:
df_mq_20k = evaluate(df_mq,model_20k_85 )

digit_by_digit_accuracy 0.664
mean_absolute_error 38.8216
weighted_digit_by_digit_accuracy 0.660952380952381
summation difference : prediction - origin 683.18
total origin consumption 1858.91
Overall error rate 0.36751644781081383


In [0]:
df_mq_10k = evaluate(df_mq,model_10k )

digit_by_digit_accuracy 0.664
mean_absolute_error 29.627200000000002
weighted_digit_by_digit_accuracy 0.6685714285714286
summation difference : prediction - origin 558.34
total origin consumption 1858.91
Overall error rate 0.30035881242233353


## LQ digital data

In [0]:
df_lq = prepareLabelDataset(df_digital = df_digital, df_label = lq_digital)
df_lq.head()

,file_name,label,quality,original_label,len_label,len_label_decimal,final_label,pictures,weights
24,0cfd795fed59753d223f222934728e9a788dd9ba.jpg,x8401,L,84.01,5,2,"[4, 10, 8, 4, 0, 1]","[[[216.0, 216.0, 216.0], [216.0, 216.0, 216.0]...",4
25,0a1e1c676aa31a9f56818e580d7a2e20689fefba.jpg,18200,L,182.00,6,2,"[5, 1, 8, 2, 0, 0]","[[[207.0, 218.0, 214.0], [205.0, 216.0, 212.0]...",5
26,0e403444174002e29f29f6805f8aaf1a503340d8.jpg,xx300,L,30.0,4,1,"[3, 10, 10, 3, 0, 0]","[[[202.0, 202.0, 202.0], [202.0, 202.0, 202.0]...",3
27,0ca648839030b87b21ae76d9685922d29a513947.jpg,x5821,L,58.21,5,2,"[4, 10, 5, 8, 2, 1]","[[[183.0, 181.0, 169.0], [183.0, 181.0, 169.0]...",4
28,0a737aefecf9233d8450aa67a6df9035c7e51bd8.jpg,x4260,L,426.0,5,1,"[4, 10, 4, 2, 6, 0]","[[[248.0, 248.0, 248.0], [249.0, 249.0, 249.0]...",4


In [0]:
df_lq_20k = evaluate(df_lq,model_20k_85 )

digit_by_digit_accuracy 0.6500000000000001
mean_absolute_error 37.18333333333334
weighted_digit_by_digit_accuracy 0.6535031847133758
summation difference : prediction - origin -50.939999999999955
total origin consumption 4077.2699999999995
Overall error rate -0.012493653841908915


In [0]:
df_lq_10k = evaluate(df_lq,model_10k )

digit_by_digit_accuracy 0.6833333333333336
mean_absolute_error 37.77111111111111
weighted_digit_by_digit_accuracy 0.6815286624203821
summation difference : prediction - origin -74.02000000000002
total origin consumption 4077.2699999999995
Overall error rate -0.018154304228074184


In [0]:
df_all_10k.to_csv("/content/drive/My Drive/Team Bergere/Optimizer/digital_prediction.csv")